## Nitish Interview Questions

Overview of this workbook:

You have an excel sheet with information about brain region cell counts.
We have injected some pretty colors to label projections. 

It has the .png file name that has information about the 
subject (example 2_1)
brain region (example aACC)
and channel (example mCherry is just mCherry and mCherry_GFP is both)
and count (number)

This means for the row for slide FF_S_S_brainregion_1_1_GFP.png has a count of cells that are positive for GFP ONLY 
specifically these cells have only GFP and nothing else therefore they should be counted as GFP only cells (or whatever GFP maps to for that subject)


Behaviorally you have two conditions, which are in two different excel sheets in the excel file.

full_focus1 is 10 min familiar (condition 1) - 25 images with one .png for every channel and channel overlay
full_focus2 is cagemate (condition 2) - 28 images with one .png for every channel and channel overlay


In [1]:
import pandas as pd

# Load the Excel file
file_path = "\Copy of cell_counts_full.xlsx"

excel_file = pd.ExcelFile(file_path)

# Print the sheet names to see what they are
print(excel_file.sheet_names)

#full_focus1 = 10min familiar condition
#full_focus2 = cagemate condition

['full_focus1', 'full_focus2', 'standard resolution ', 'QC']


In [2]:
# Load both sheets into separate DataFrames
sheet = pd.read_excel(file_path)

# Rename columns for consistency
sheet1.columns = ["Slice", "Count", "Total Area", "Average Size", "%Area", "Mean", "Perim.", "Threshold"]
sheet2.columns = ["Slice", "Count", "Total Area", "Average Size", "%Area", "Mean", "Perim.", "Threshold"]

# Drop the 'Threshold' column
sheet1 = sheet1.drop(columns=['Threshold','Total Area',"Average Size", "%Area", "Mean", "Perim."])
sheet2 = sheet2.drop(columns=['Threshold','Total Area',"Average Size", "%Area", "Mean", "Perim."])

# Drop all rows with NaN values
sheet1 = sheet1.dropna()
sheet2 = sheet2.dropna()

# Define subject-specific mappings for Channel in full_focus1
subject_mappings_full_focus1 = {
    '2_1': {'GFP': 'NAc', 'mCherry': 'MD'},
    '2_2': {'GFP': 'NAc', 'mCherry': 'MD'},
    '2_3': {'GFP': 'NAc', 'mCherry': 'MD'},
    '1_2': {'GFP': 'MD', 'mCherry': 'NAc'},
    '1_3': {'GFP': 'MD', 'mCherry': 'NAc'}
}

# Define the default mapping for full_focus2
default_mapping_full_focus2 = {'GFP': 'MD', 'mCherry': 'NAc'}

# Function to extract Channel from Slice
def extract_channel(slice_value):
    if 'GFP' in slice_value:
        return 'GFP'
    elif 'mCherry' in slice_value:
        return 'mCherry'
    elif 'cFos' in slice_value:
        return 'cFos'
    return 'Unknown'

# Function to map the channel based on subject
def map_channel(slice_value, channel_value, sheet_name):
    parts = slice_value.split('_')
    subject = parts[2]  # Extract subject identifier from the Slice column
    if sheet_name == 'full_focus1' and subject in subject_mappings_full_focus1:
        region_mapping = subject_mappings_full_focus1[subject]
        return region_mapping.get(channel_value, 'Unknown')
    elif sheet_name == 'full_focus2':
        return default_mapping_full_focus2.get(channel_value, 'Unknown')
    return 'Unknown'

    # Define overlaps based on the presence of markers in the Slice column
def determine_overlap(slice_value):
    markers = []
    if 'cFos' in slice_value:
        markers.append('cFos')
    if 'GFP' in slice_value:
        markers.append('GFP')
    if 'mCherry' in slice_value:
        markers.append('mCherry')
    # Join markers to form the overlap label
    return '_'.join(sorted(markers))

# Function to extract markers from Slice and create projector label
def create_projector(slice_value, subject, sheet_name):
    markers = []
    if 'cFos' in slice_value:
        markers.append('cFos')
    if 'GFP' in slice_value:
        markers.append('GFP')
    if 'mCherry' in slice_value:
        markers.append('mCherry')
    if sheet_name == 'full_focus1' and subject in subject_mappings_full_focus1:
        region_mapping = subject_mappings_full_focus1[subject]
    else:
        region_mapping = default_mapping_full_focus2
    combined_markers = []
    for marker in markers:
        if marker in ['GFP', 'mCherry']:
            region = region_mapping.get(marker, 'Unknown')
            combined_markers.append(region)
        else:
            combined_markers.append(marker)
    return '+'.join(combined_markers) + '+'

# Function to extract the subject number from the Slice column
def extract_subject(slice_value):
    parts = slice_value.split('_')
    if len(parts) >= 3:
        return parts[2]  # Assuming the subject number is in the 2nd 
    return 'Unknown'  # In case the slice format is unexpected

def extract_region(slice_value):
    parts = slice_value.split('_')
    if len(parts) >= 4:
        return parts[3]  # Assuming the region information is in the 4th position
    return 'Unknown'

# Process the data for both sheets
def process_data(df, sheet_name):
    df['Slice'] = df['Slice'].astype(str)
    df['Subject'] = df['Slice'].apply(extract_subject)  # Extract the subject number
    df['Region'] = df['Slice'].apply(extract_region)    # Extract the region information
    df['Channel'] = df['Slice'].apply(extract_channel)
    df['CellType'] = df.apply(lambda row: map_channel(row['Slice'], row['Channel'], sheet_name), axis=1)
    df['Overlap'] = df['Slice'].apply(determine_overlap)
    df['Projector'] = df.apply(lambda row: create_projector(row['Slice'], row['Subject'], sheet_name), axis=1)
    return df
# Function to extract the region information from the Slice column
def extract_region(slice_value):
    parts = slice_value.split('_')
    if len(parts) >= 4:
        return parts[3]  # Assuming the region information is in the 4th position
    return 'Unknown'  # In case the slice format is unexpected


# Apply the updated function to both sheets
sheet1_processed = process_data(sheet1, 'full_focus1')
sheet2_processed = process_data(sheet2, 'full_focus2')

# Print the processed data (or save to new Excel file, etc.)
print("Processed data from full_focus1:")
print(sheet1_processed)

print("Processed data from full_focus2:")
print(sheet2_processed)


Processed data from full_focus1:
                                    Slice  Count Subject Region  Channel  \
0                FF_2_2_aACC_1_1_cFos.png     96     2_2   aACC     cFos   
1                 FF_2_2_aACC_1_1_GFP.png     96     2_2   aACC      GFP   
2             FF_2_2_aACC_1_1_mCherry.png    100     2_2   aACC  mCherry   
3         FF_2_2_aACC_1_1_mCherry_GFP.png     39     2_2   aACC      GFP   
4    FF_2_2_aACC_1_1_mCherry_GFP_cFos.png     40     2_2   aACC      GFP   
..                                    ...    ...     ...    ...      ...   
170                FF_2_3_mIL_6_1_GFP.png    387     2_3    mIL      GFP   
171           FF_2_3_mIL_6_1_GFP_cFos.png    450     2_3    mIL      GFP   
172        FF_2_3_mIL_6_1_mCherry_GFP.png    431     2_3    mIL      GFP   
173   FF_2_3_mIL_6_1_mCherry_GFP_cFos.png    385     2_3    mIL      GFP   
174       FF_2_3_mIL_6_1_mCherry_cFos.png    410     2_3    mIL  mCherry   

    CellType           Overlap     Projector  
0    Un

In [3]:
example_slice = 'FF_2_2_aACC_1_1_mCherry_GFP_cFos.png'
example_row = sheet1_processed[sheet1_processed['Slice'] == example_slice]
if not example_row.empty:
    example_projector = example_row['Projector'].values[0]
    print(f"Projector for {example_slice}: {example_projector}")

Projector for FF_2_2_aACC_1_1_mCherry_GFP_cFos.png: cFos+NAc+MD+


BUG: 
For full_focus2 the counts are correct. 

For the full_focus1 the counts are off. Specifically for multiple projectors 
full_focus1 totals: 
   - MD+ = 7224
   - NAc+ = 6049
   - NAc+MD+ = 4670
   - cFos = 5308
   - cFos+MD+ = 5080
   - cFos+NAc+ = 4641
   - cFos+MD+NAc+ = 4166 

In [4]:
projector_counts = pd.concat([sheet1_processed.groupby('Projector')['Count'].sum().reset_index().assign(sheet='full_focus1'),
                             sheet2_processed.groupby('Projector')['Count'].sum().reset_index().assign(sheet='full_focus2')])

print("Total counts for each projector by sheet:")
print(projector_counts)

Total counts for each projector by sheet:
      Projector  Count        sheet
0           MD+   7224  full_focus1
1          NAc+   6147  full_focus1
2       NAc+MD+   4833  full_focus1
3         cFos+   5308  full_focus1
4      cFos+MD+   4641  full_focus1
5     cFos+NAc+   4982  full_focus1
6  cFos+NAc+MD+   4003  full_focus1
0           MD+   3451  full_focus2
1       MD+NAc+   2378  full_focus2
2          NAc+   4799  full_focus2
3         cFos+   4309  full_focus2
4      cFos+MD+   2786  full_focus2
5  cFos+MD+NAc+   2423  full_focus2
6     cFos+NAc+   3079  full_focus2
